In [26]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from scipy.sparse import hstack
from sklearn.pipeline import FeatureUnion

In [62]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn

In [47]:
from collections import Counter, OrderedDict


In [71]:
import joblib


In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [6]:
train=pd.read_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/train.csv')

In [27]:
test=pd.read_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/test.csv')

In [28]:
test.head()

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة ...,neutral,حقوق المراه التي تضمنها لها وزاره العدل,['MojKsa'],7,False,NaN,['https://t.co/QUGzWwubFk']
1,4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او...,neutral,ولي امر الزوجه او ولي الزوجه او ولي المراه من ...,['abm112211'],47,False,NaN,['https://t.co/rkqxvm5qY7']
2,8,مبدائيا انا مع حقوق المرأة وضد حقوق الرجل 😁,neutral,مبدائيا انا مع حقوق المراه وضد حقوق الرجل,[],8,True,:beaming_face_with_smiling_eyes:,[]
3,19,RT @Stranger23_: @Oh__28 @LifeRights19 عشان كذ...,negative,عشان كذا انا عندي تحفظ للي يشوف الغاء الولايه ...,"['Stranger23', 'Oh', 'LifeRights19']",49,True,:downcast_face_with_sweat:,[]
4,34,@Eqbal_Darandari @g11_we المفروض تتم الموافقة ...,neutral,المفروض تتم الموافقه وبلاجماع مافي ابدا توافق ...,"['Eqbal', 'g11']",25,False,NaN,[]


In [7]:
train.head()

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,['https://t.co/Mzf90Ta5g1']
1,2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"['saud', 'Shubratanmyeh']",15,True,:sparkles:,['https://t.co/c2NXzNCdLU']
2,5,RT @fatemaar7: انا كامرأة يوجعني كل حزن نساء ا...,negative,انا كامراه يوجعني كل حزن نساء العالم سواء تعنف...,['fatemaar7'],50,False,NaN,['https://t.co/c494qBVPx4']
3,7,@AliFCD @ShamsanM @AlyemenNor @space_dist @m2r...,negative,غسق وسبيس وحلوه الحلوات هم اشخاص لهم هويتهم وك...,"['AliFCD', 'ShamsanM', 'AlyemenNor', 'space', ...",32,False,NaN,[]
4,9,RT @Dresraamohammed: السؤال بقى للناس الي شغال...,negative,السؤال بقي للناس الي شغاله في حقوق الانسان حقو...,['Dresraamohammed'],24,True,:person_facepalming_light_skin_tone::female_si...,[]


In [29]:
def apply_tf_idf_cnt_vec(train,test):
  union = FeatureUnion([("tf_idf_ngra_3", TfidfVectorizer(min_df=1,ngram_range=(1, 3),binary=True,max_features=700)),
                    ( "tf_idf_ngra_2", TfidfVectorizer(min_df=1,ngram_range=(1, 2),binary=True,max_features=700)),
                  ("tf_idf_ngra_1",TfidfVectorizer(min_df=1,ngram_range=(1, 1),binary=True,max_features=700)),
                  ("tf_idf_ngra_5", TfidfVectorizer(min_df=1,ngram_range=(1, 5),binary=True,max_features=700)),
                  ("tf_idf_ngra_7", TfidfVectorizer(min_df=1,ngram_range=(1, 7),binary=True,max_features=700)),
                  ("tf_idf_ngra_3_ch",TfidfVectorizer(min_df=1,ngram_range=(1, 3),binary=True,max_features=700,analyzer='char')),
                  ("tf_idf_ngra_2_ch",TfidfVectorizer(min_df=1,ngram_range=(1, 2),binary=True,max_features=700,analyzer='char')),
                  ("tf_idf_ngra_1_ch",TfidfVectorizer(min_df=1,ngram_range=(1, 1),binary=True,max_features=700,analyzer='char')),
                  ("tf_idf_ngra_5_ch",TfidfVectorizer(min_df=1,ngram_range=(1, 5),binary=True,max_features=700,analyzer='char')),
                  ("tf_idf_ngra_7_ch", TfidfVectorizer(min_df=1,ngram_range=(1, 7),binary=True,max_features=700,analyzer='char')),
                  ("cnt_vec_ngra_3", CountVectorizer(min_df=1,ngram_range=(1, 3),binary=True,max_features=700)),
                  ("cnt_vec_ngra_5", CountVectorizer(min_df=1,ngram_range=(1, 5),binary=True,max_features=700))
                                  
                                  
                                  ])
  train_feat_vec=union.fit_transform(train)
  test_feat_vec=union.transform(test)
  return train_feat_vec,test_feat_vec
                



In [31]:
train.dropna(inplace=True)

In [32]:
test.dropna(inplace=True)

In [33]:
train_vec,test_vec=apply_tf_idf_cnt_vec(train['cleaned_text'],test['cleaned_text'])

In [72]:
def print_statistics(y, y_pred):
    accuracy = metrics.accuracy_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average='weighted')
    recall = metrics.recall_score(y, y_pred, average='weighted')
    f_score = metrics.f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(metrics.classification_report(y, y_pred))
    return accuracy, precision, recall, f_score



def plot_coefficients(classifier, feature_names, top_features=20, plot_name="/bow_models/bow_binary_"):
    # Get the top most positive/negative coefficients
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    x_names = [feature_names[feature] for feature in top_coefficients]

    # Plot the coefficients
    plt.figure(figsize=(15, 5))
    colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
    plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    plt.xticks(np.arange(0, 2 * top_features), x_names, rotation=30, ha='right')
    plt.ylabel("Coefficient Value")
    plt.title("Visualising the top %d features taken up by an SVM model" % top_features)
    to_save_filename = path + "/plots/" + plot_name + "top%d_coefficients.png" % top_features
    plt.savefig(to_save_filename)
    print("Coefficients' visualisation saved to %s\n" % to_save_filename)

def get_regularization_params(a=-1, b=1, c=3, d=1, e=5):
    reg_range = np.outer(np.logspace(a, b, c), np.array([d, e]))
    reg_range = reg_range.flatten()
    return reg_range


def grid_classifier(x_train, y_train, x_test, y_test, model, parameters,
                    make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    grid = GridSearchCV(estimator=model, param_grid=parameters, verbose=0)
    grid.fit(x_train, y_train)
    sorted(grid.cv_results_.keys())
    classifier = grid.best_estimator_
    if make_feature_analysis:
        plot_coefficients(classifier, feature_names, top_features, plot_name)
    y_hat = classifier.predict(x_test)
    print_statistics(y_test, y_hat)

# Method to print the header of the currently running model
def print_model_title(name):
    print("\n==================================================================")
    print('{:>20}'.format(name))
    print("==================================================================\n")


def linear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
               make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Linear SVM")
    C_range = get_regularization_params()
    parameters = {'C': C_range}
    linear_svm = LinearSVC(C=1.0, class_weight=class_ratio, penalty='l2')
    grid_classifier(x_train, y_train, x_test, y_test, linear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def nonlinear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
                  make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Nonlinear SVM")
    C_range = get_regularization_params(a=-1, b=0, c=2, d=1, e=5)
    gamma_range = get_regularization_params(a=-2, b=-1, c=2, d=1, e=5)
    parameters = {'kernel': ['rbf'], 'C': C_range, 'gamma': gamma_range}
    nonlinear_svm = SVC(class_weight=class_ratio)
    grid_classifier(x_train, y_train, x_test, y_test, nonlinear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def logistic_regression_grid(x_train, y_train, x_test, y_test, class_ratio,
                        make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Logistic Regression")
    C_range = [0.001, 0.01, 0.1, 1, 10, 100]
    parameters = {'C': C_range}
    log_regr = LogisticRegression(C=1.0, class_weight=class_ratio, penalty='l2')
    grid_classifier(x_train, y_train, x_test, y_test, log_regr, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def linear_svm(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Linear SVM")
    svm = LinearSVC(C=0.01, class_weight=class_ratio, penalty='l2')
    svm.fit(x_train, y_train)
    y_hat = svm.predict(x_test)
    print_statistics(y_test, y_hat)
    filename = '/content/drive/MyDrive/Omdena_sentiment/Saved_models/svm_model.sav'
    joblib.dump(svm, filename)


def logistic_regression(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Logistic Regression")
    regr = LogisticRegression(C=0.01, class_weight=class_ratio, penalty='l2')
    regr.fit(x_train, y_train)
    y_hat = regr.predict(x_test)
    print_statistics(y_test, y_hat)
    filename = '/content/drive/MyDrive/Omdena_sentiment/Saved_models/lr_model.sav'
    joblib.dump(regr, filename)


def feature_selection(x_train, y_train, x_test, y_test):
    print("Feature selection with LinearSVC")
    model = LinearSVC(C=0.1, penalty='l2')
    rfe = RFE(model, 5)
    best_features_model = rfe.fit(x_train, y_train)
    y_hat = best_features_model.predict(x_test)
    print_statistics(y_test, y_hat)



In [73]:
linear_svm(train_vec,train['Class_camel'],test_vec,test['Class_camel'])



          Linear SVM

Accuracy: 0.739
Precision: 0.744
Recall: 0.739
F_score: 0.741

              precision    recall  f1-score   support

    negative       0.69      0.75      0.72      5607
     neutral       0.63      0.65      0.64      2266
    positive       0.81      0.76      0.79      7761

    accuracy                           0.74     15634
   macro avg       0.71      0.72      0.71     15634
weighted avg       0.74      0.74      0.74     15634



In [74]:
logistic_regression(train_vec,train['Class_camel'],test_vec,test['Class_camel'])



 Logistic Regression

Accuracy: 0.692
Precision: 0.717
Recall: 0.692
F_score: 0.697

              precision    recall  f1-score   support

    negative       0.65      0.73      0.69      5607
     neutral       0.50      0.69      0.58      2266
    positive       0.83      0.67      0.74      7761

    accuracy                           0.69     15634
   macro avg       0.66      0.69      0.67     15634
weighted avg       0.72      0.69      0.70     15634

